In [1]:
#!pip install --ignore-installed PyYAML
#!pip install transformers
#!pip install tensorflow
#!pip install datasets

# Train a model from scratch (RobertaForMaskedLM)

## Load dataset

In [14]:
from datasets import load_dataset

dataset = load_dataset('oscar', 'unshuffled_deduplicated_it')

dataset['train'][0]

Reusing dataset oscar (/home/xiaopengxu/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_it/1.0.0/e4f06cecc7ae02f7adf85640b4019bf476d44453f251a1d84aebae28b0f8d51d)


{'id': 0,
 'text': "La estrazione numero 48 del 10 e LOTTO ogni 5 minuti e' avvenuta sabato 15 settembre 2018 alle ore 04:00 a Roma, nel Centro Elaborazione Dati della Lottomatica Italia (ora GTech SpA), con la supervisione della Amministrazione Autonoma dei Monopoli di Stato (AAMS), incaricata di vigilare sulla regolarità delle operazioni di sorteggio.\nIl Montepremi della 48ª estrazione viene ripartito tra i vincitori delle singole categorie di premio.\nRicorda di controllare il Numero ORO 53. E, se lo hai giocato, anche il DOPPIO ORO 53 e 66. Se indovini puoi vincere premi più ricchi.\nIl nostro sito web impiega cookies per migliorare la navigazione del visitatore. L’utente è consapevole che, continuando a visitare il nostro sito web, accetta l’utilizzo dei cookies Accetto Informazioni\n(C) Copyright 2013-2017 10elotto.biz | Il presente sito è da considerarsi un sito indipendente, NON collegato alla rete ufficiale Gtech SpA."}

In [16]:
#! ls -alh *

### Save data to train tokenizer

In [17]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 10K mark, save to file
        with open(f'./oscar_it/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'./oscar_it/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

## Building a Tokenizer

In [18]:
from pathlib import Path
paths = [str(x) for x in Path('./oscar_it').glob('**/*.txt')]

In [19]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [20]:
tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

### Save tokenizer

In [23]:
import os

os.mkdir('./filiberto')

tokenizer.save_model('filiberto')

['filiberto/vocab.json', 'filiberto/merges.txt']

### Initializing the Tokenizer

In [24]:
from transformers import RobertaTokenizer

# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('filiberto', max_len=512)

In [25]:
# test our tokenizer on a simple sentence
tokens = tokenizer('ciao, come va?')
print(tokens)

{'input_ids': [0, 13722, 16, 479, 600, 35, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


## Creating the Input Pipeline
### Preparing the Data

In [27]:
with open('./oscar_it/text_0.txt', 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')

In [28]:
batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
len(batch)

2

### Building the DataLoader

In [29]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

NameError: name 'input_ids' is not defined

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [ ]:
dataset = Dataset(encodings)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

## Training the Model

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

### Training Preparation

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

### Training

In [ ]:
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
model.save_pretrained('./filiberto')  # and don't forget to save filiBERTo!

## The Real Test

In [ ]:
from transformers import pipeline

fill = pipeline('fill-mask', model='filiberto', tokenizer='filiberto')
fill(f'ciao {fill.tokenizer.mask_token} va?')

In [ ]:
fill(f'buongiorno, {fill.tokenizer.mask_token} va?')

In [ ]:
fill(f'ciao, dove ci {fill.tokenizer.mask_token} oggi pomeriggio? ')

In [ ]:
fill(f'cosa sarebbe successo se {fill.tokenizer.mask_token} scelto un altro giorno?') # a harder sentence